# California housing dataset regression with decision trees 

In this notebook, we'll use [decision trees](http://scikit-learn.org/stable/modules/tree.html) and [ensembles of trees](http://scikit-learn.org/stable/modules/ensemble.html) to estimate median house values on Californian housing districts using scikit-learn and [XGBoost](https://xgboost.readthedocs.io/en/latest/).

First, the needed imports. 

In [ ]:
%matplotlib inline

import numpy as np
from sklearn import datasets, __version__
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Data

Then we load the California housing data. First time we need to download the data, which can take a while.

In [ ]:
chd = datasets.fetch_california_housing()

We'll split the data into a training and a test set.

Let's also select a single attribute to start the analysis with, say *MedInc*.

In [ ]:
test_size = 5000
single_attribute = 'MedInc'

X_train_all, X_test_all, y_train, y_test = train_test_split(
    chd.data, chd.target, test_size=test_size, shuffle=True)

attribute_index = chd.feature_names.index(single_attribute)
X_train_single = X_train_all[:, attribute_index].reshape(-1, 1)
X_test_single = X_test_all[:, attribute_index].reshape(-1, 1)
     
print()
print('California housing data: train:',len(X_train_all),'test:',len(X_test_all))
print()
print('X_train_all:', X_train_all.shape)
print('X_train_single:', X_train_single.shape)
print('y_train:', y_train.shape)
print()
print('X_test_all', X_test_all.shape)
print('X_test_single', X_test_single.shape)
print('y_test', y_test.shape)

The training data matrix `X_train_all` is a matrix of size (`n_train`, 8), and `X_train_single` contains only the first attribute (*MedInc* by default). The vector `y_train` contains the target value (median house value) for each housing district in the training set.

Let's start our analysis with the single attribute. Later, you can set `only_single_attribute = False` to use all eight attributes in the regression.

In [ ]:
only_single_attribute = True

if only_single_attribute:
    X_train = X_train_single
    X_test = X_test_single
else:
    X_train = X_train_all
    X_test = X_test_all

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)

## Decision tree

Decision tree is a model that predicts the value of a target variable by learning simple *if-then-else* decision rules inferred from the data features.

### Learning

The parameter `max_depth` specifies the maximum depth of the tree.

In [ ]:
%%time

max_depth = 3
dt_reg = DecisionTreeRegressor(max_depth=max_depth)
dt_reg.fit(X_train, y_train)

We can visualize the results if we are using only a single attribute:

In [ ]:
if X_train.shape[1] == 1:
    plt.figure(figsize=(10, 10))
    plt.scatter(X_train, y_train, s=5)
    reg_x = np.arange(np.min(X_train), np.max(X_train), 0.01).reshape(-1, 1)
    plt.plot(reg_x, dt_reg.predict(reg_x), lw=4, c=sns.color_palette()[1],
             label='decision tree')
    plt.legend(loc='best');

### Inference

We use *mean squared error* as the performance measure for our regression algorihm: 

In [ ]:
%%time

predictions = dt_reg.predict(X_test)
print("Mean squared error: %.3f"
      % mean_squared_error(y_test, predictions))

## Random forest

Random forest is an ensemble (or a group; hence the name *forest*) of decision trees, obtained by introducing randomness into the tree generation. The prediction of the random forest is obtained by *averaging* the predictions of the individual trees.

Random forest is a solid workhorse that almost always produces serviceable results without much tuning.

### Learning

Random forest classifiers are quick to train, quite robust to hyperparameter values, and often work relatively well.

In [ ]:
%%time

n_estimators = 10
max_depth = 3
rf_reg = RandomForestRegressor(n_estimators=n_estimators,
                               max_depth=max_depth)
rf_reg.fit(X_train, y_train)

In [ ]:
if X_train.shape[1] == 1:
    plt.figure(figsize=(10, 10))
    plt.scatter(X_train, y_train, s=5)
    reg_x = np.arange(np.min(X_train), np.max(X_train), 0.01).reshape(-1, 1)
    plt.plot(reg_x, dt_reg.predict(reg_x), lw=4, c=sns.color_palette()[1],
             label='decision tree')
    plt.plot(reg_x, rf_reg.predict(reg_x), lw=4, c=sns.color_palette()[2],
             label='random forest')
    plt.legend(loc='best');

### Inference

In [ ]:
%%time

predictions = rf_reg.predict(X_test)
print("Mean squared error: %.3f"
      % mean_squared_error(y_test, predictions))

## Gradient boosted trees (XGBoost)

Gradient boosted trees (or extreme gradient boosted trees) is another way of constructing ensembles of decision trees, using the *boosting* framework.  Let's use a popular separate package, [XGBoost](http://xgboost.readthedocs.io/en/latest/), to train gradient boosted trees for regression.  

XGBoost has been recently used to obtain record-breaking results on many machine learning competitions, but have quite a lot of hyperparameters that need to be carefully tuned to get the best performance.

### Learning

In [ ]:
%%time

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

In [ ]:
if X_train.shape[1] == 1:
    plt.figure(figsize=(10, 10))
    plt.scatter(X_train, y_train, s=5)
    reg_x = np.arange(np.min(X_train), np.max(X_train), 0.01).reshape(-1, 1)
    plt.plot(reg_x, dt_reg.predict(reg_x), lw=4, c=sns.color_palette()[1],
             label='decision tree')
    plt.plot(reg_x, rf_reg.predict(reg_x), lw=4, c=sns.color_palette()[2],
             label='random forest')
    plt.plot(reg_x, xgb_reg.predict(reg_x), lw=4, c=sns.color_palette()[3],
             label='XGBoost')
    plt.legend(loc='best');

### Inference

In [ ]:
%%time

predictions = xgb_reg.predict(X_test)
print("Mean squared error: %.3f"
      % mean_squared_error(y_test, predictions))

## Model tuning

Study the documentation of the different decision tree models used in this notebook ([decision trees](http://scikit-learn.org/stable/modules/tree.html), [tree ensembles](http://scikit-learn.org/stable/modules/ensemble.html), [XGBoost](https://xgboost.readthedocs.io/en/latest/)), and experiment with different hyperparameter values.  

Report the lowest mean squared error you manage to obtain for each model type.  Also mark down the parameters you used, so others can try to reproduce your results. 